<a href="https://colab.research.google.com/github/ShreshthaAggarwal155/ML-Lab_C3/blob/main/Lab_6/Bayesian_Network(with_output).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
!pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.9 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD
from IPython.display import Image

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv("data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        297 non-null    int64  
 1   sex        297 non-null    int64  
 2   cp         297 non-null    int64  
 3   trestbps   297 non-null    int64  
 4   chol       297 non-null    int64  
 5   fbs        297 non-null    int64  
 6   restecg    297 non-null    int64  
 7   thalach    297 non-null    int64  
 8   exang      297 non-null    int64  
 9   oldpeak    297 non-null    float64
 10  slope      297 non-null    int64  
 11  ca         297 non-null    int64  
 12  thal       297 non-null    int64  
 13  condition  297 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 32.6 KB


In [ ]:
df.corr()["condition"]

age          0.227075
sex          0.278467
cp           0.408945
trestbps     0.153490
chol         0.080285
fbs          0.003167
restecg      0.166343
thalach     -0.423817
exang        0.421355
oldpeak      0.424052
slope        0.333049
ca           0.463189
thal         0.520516
condition    1.000000
Name: condition, dtype: float64

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="cp", color='cp',pattern_shape="condition", title="Chest pain type VS Heart Disease")
fig.show()

In [ ]:
fig = px.histogram(df, x="thal", color='thal',pattern_shape="condition", title="Thal VS Heart Disease",text_auto=True)
fig.show()

In [ ]:
Model=BayesianNetwork([
                     ('age','trestbps'),('age','fbs'),
                     ('sex','trestbps'),
                     ('exang','trestbps'),
                     ('thal','condition'),
                     ('trestbps','condition'),
                     ('fbs','condition'),
                     ('condition','restecg'),('condition','thalach'),('condition','chol')
                     ])

def HeartDisease(x):
    prediction =  "No Heart Disease" if x[0] > x[1]  else "Heart Disease"
    print("\n Its predicted that the patient has " + prediction)

In [ ]:
print('\n Learning CPD using Maximum likelihood estimators')
Model.fit(df,estimator=MaximumLikelihoodEstimator)


 Learning CPD using Maximum likelihood estimators


In [ ]:
HeartDisease_infer = VariableElimination(Model)

In [ ]:
print('\n 1. Probability of HeartDisease given Age=30')
q=HeartDisease_infer.query(variables=['condition'],evidence={'age':28})
print(q)
HeartDisease(q.values)


 1. Probability of HeartDisease given Age=30


/usr/local/lib/python3.7/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning:

Found unknown state name. Trying to switch to using all state names as state numbers



  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

+--------------+------------------+
| condition    |   phi(condition) |
+==============+==================+
| condition(0) |           0.6060 |
+--------------+------------------+
| condition(1) |           0.3940 |
+--------------+------------------+

 Its predicted that the patient has No Heart Disease


In [ ]:
print('\n 2. Probability of HeartDisease given cholesterol=100')
q=HeartDisease_infer.query(variables=['condition'],evidence={'chol':100})
print(q)
HeartDisease(q.values)


 2. Probability of HeartDisease given cholesterol=100


/usr/local/lib/python3.7/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning:

Found unknown state name. Trying to switch to using all state names as state numbers



  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

+--------------+------------------+
| condition    |   phi(condition) |
+==============+==================+
| condition(0) |           1.0000 |
+--------------+------------------+
| condition(1) |           0.0000 |
+--------------+------------------+

 Its predicted that the patient has No Heart Disease


In [ ]:
print('\n 3. Probability of HeartDisease given cholesterol, thal')
q=HeartDisease_infer.query(variables=['condition'],evidence={'chol':282,'thal':2})
print(q)
HeartDisease(q.values)


 3. Probability of HeartDisease given cholesterol, thal


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

+--------------+------------------+
| condition    |   phi(condition) |
+==============+==================+
| condition(0) |           0.0000 |
+--------------+------------------+
| condition(1) |           1.0000 |
+--------------+------------------+

 Its predicted that the patient has Heart Disease


In [ ]:
#2nd case

EQ_model = BayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "JohnCalls"),
        ("Alarm", "MaryCalls"),
    ]
)

In [ ]:
cpd_B = TabularCPD(variable="Burglary", variable_card=2, values=[[0.001], [1 - 0.001]])

cpd_E = TabularCPD(variable="Earthquake", variable_card=2, values=[[0.002], [1 - 0.002]])

cpd_A = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.95, 0.94, 0.29, 0.001], [1 - 0.95, 1 - 0.94, 1 - 0.29, 1 - 0.001]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)

cpd_J = TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[0.90, 0.05], [0.10, 0.95]],
    evidence=["Alarm"],
    evidence_card=[2],
)

cpd_M = TabularCPD(
    variable="MaryCalls",
    variable_card=2,
    values=[[0.70, 0.01], [0.30,0.99]],
    evidence=["Alarm"],
    evidence_card=[2],
)

In [ ]:
EQ_model.add_cpds(cpd_B, cpd_E, cpd_A, cpd_J, cpd_M)

EQ_model.check_model()

True

In [ ]:
print(EQ_model.get_cpds("Burglary"))
print(EQ_model.get_cpds("Earthquake"))
print(EQ_model.get_cpds("Alarm"))
print(EQ_model.get_cpds("JohnCalls"))
print(EQ_model.get_cpds("MaryCalls"))
print("Nodes in the model:", EQ_model.nodes())

+-------------+-------+
| Burglary(0) | 0.001 |
+-------------+-------+
| Burglary(1) | 0.999 |
+-------------+-------+
+---------------+-------+
| Earthquake(0) | 0.002 |
+---------------+-------+
| Earthquake(1) | 0.998 |
+---------------+-------+
+------------+----------------------+-----+---------------+---------------+
| Burglary   | Burglary(0)          | ... | Burglary(1)   | Burglary(1)   |
+------------+----------------------+-----+---------------+---------------+
| Earthquake | Earthquake(0)        | ... | Earthquake(0) | Earthquake(1) |
+------------+----------------------+-----+---------------+---------------+
| Alarm(0)   | 0.95                 | ... | 0.29          | 0.001         |
+------------+----------------------+-----+---------------+---------------+
| Alarm(1)   | 0.050000000000000044 | ... | 0.71          | 0.999         |
+------------+----------------------+-----+---------------+---------------+
+--------------+----------+----------+
| Alarm        | Alarm(0) |

In [ ]:
print(EQ_model.is_dconnected("Burglary", "Earthquake"))
print(EQ_model.is_dconnected("Burglary", "Earthquake", observed=["Alarm"]))

False
True


In [ ]:
EQ_infer = VariableElimination(EQ_model)

In [ ]:
q = EQ_infer.query(variables=["MaryCalls"], evidence={"Alarm":0})
print(q)
output = "Yes " if q.values[0] > q.values[1] else "No"
print('\n' +output)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+--------------+------------------+
| MaryCalls    |   phi(MaryCalls) |
+==============+==================+
| MaryCalls(0) |           0.7000 |
+--------------+------------------+
| MaryCalls(1) |           0.3000 |
+--------------+------------------+

Yes 


In [ ]:
print("Probability of burglary given both of them call\n")
q = EQ_infer.query(variables=["Burglary"], evidence={"MaryCalls":0,"JohnCalls":0})
print(q)
output = "Yes " if q.values[0] > q.values[1] else "No"
print('\n' +output)

Probability of burglary given both of them call



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.2842 |
+-------------+-----------------+
| Burglary(1) |          0.7158 |
+-------------+-----------------+

No


In [ ]:
print("Probability of Alarm given both of them call\n")
q = EQ_infer.query(variables=["Alarm"], evidence={"MaryCalls":0,"JohnCalls":0})
print(q)
output = "Yes " if q.values[0] > q.values[1] else "No"
print('\n' +output)

Probability of Alarm given both of them call



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.7607 |
+----------+--------------+
| Alarm(1) |       0.2393 |
+----------+--------------+

Yes 
